# SDSS: About the Data

In [1]:
import re
import pandas as pd
from IPython.core.display import HTML
HTML(open("styles/stylesheet.css", "r").read())

This notebook explains the dataset **`data/sdss_dr7_photometry.csv.gz`**. This dataset contains 2.8 million objects that have been spectroscopically idendified in the <a href="http://classic.sdss.org/dr7/" target="_blank">SDSS Data Release 7</a>. Our goal is to build a classifier which can predict whether an object is a galaxy, a star, or a quasar, based on the photometric measurements. As an example, here are the first five objects:

In [2]:
sdss = pd.io.parsers.read_csv("data/sdss_dr7_photometry.csv.gz", compression="gzip", index_col=["ra", "dec"])
sdss.head()

,,class,subclass,redshift,redshiftErr,psfMag_u,psfMagErr_u,psfMag_g,psfMagErr_g,psfMag_r,psfMagErr_r,...,petroMag_g,petroMagErr_g,petroMag_r,petroMagErr_r,petroMag_i,petroMagErr_i,petroMag_z,petroMagErr_z,petroRad_r,petroRadErr_r
ra,dec,,,,,,,,,,,,,,,,,,,,,
94.799215,-0.718034,Star,F5,0.000210,0.000012,18.34817,0.019610,16.99189,0.009549,16.51436,0.007483,...,16.84973,0.005293,16.33935,0.005752,16.09515,0.005556,15.96056,0.008399,1.545161,0.039029
94.807071,-0.784311,Star,F9,0.000339,0.000010,19.15451,0.027677,17.61224,0.014451,16.93648,0.011777,...,17.47206,0.025140,16.72827,0.021432,16.42008,0.022109,16.27369,0.025590,1.613344,0.029897
239.112090,-0.021816,Galaxy,STARFORMING,0.042336,0.000006,20.10414,0.039358,19.08372,0.029395,18.58049,0.035370,...,18.19883,0.062008,17.64275,0.074973,17.29095,0.083256,17.20502,0.056375,3.419085,0.120573
239.092517,0.371069,Galaxy,null,0.126354,0.000027,21.73540,0.125726,19.87871,0.018848,18.67026,0.018125,...,18.44894,0.031190,17.44512,0.027104,16.96356,0.022789,16.61036,0.059611,6.414225,1.277010
239.162223,0.351117,Galaxy,STARFORMING,0.032495,0.000016,21.11542,0.077430,19.66515,0.018378,18.75507,0.018734,...,16.32442,0.010431,15.63768,0.009456,15.27945,0.012560,14.96104,0.035006,14.036400,0.935760


The first two columns (**`ra`** and **`dec`**) are the right ascension and the declination of the object in degrees. These are the row index in our Data Frame. The third column (**`class`**) is the spectroscopic class (Star, Galaxy, and Quasar) as determined by expert opnion. This will be the target vector in the classficiation. Some objects are also further divided into subclasses.  The columns (**`redshift`** and **`redshiftErr`**) are the redshift (with errror) of the object, also determined by expert opinion.

There are 11 columns that we can use as feature vectors. These are the different <a href="https://www.sdss3.org/dr10/algorithms/magnitudes.php#mag_psf" target="_blank">PSF</a> and <a href="https://www.sdss3.org/dr10/algorithms/magnitudes.php#mag_petro" target="_blank">Petrosian</a> magnitude measurements:

* **`psfMag_u`**: PSF magnitude measurement in u-band, assuming the object is a point souce
* **`psfMag_g`**: PSF magnitude measurement in g-band, assuming the object is a point souce
* **`psfMag_r`**: PSF magnitude measurement in r-band, assuming the object is a point souce
* **`psfMag_i`**: PSF magnitude measurement in i-band, assuming the object is a point souce
* **`psfMag_z`**: PSF magnitude measurement in z-band, assuming the object is a point souce
* **`petroMag_u`**: Petrosian magnitude measurement in u-band, assuming the object is an extended souce
* **`petroMag_g`**: Petrosian magnitude measurement in g-band, assuming the object is an extended souce
* **`petroMag_r`**: Petrosian magnitude measurement in r-band, assuming the object is an extended souce
* **`petroMag_i`**: Petrosian magnitude measurement in i-band, assuming the object is an extended souce
* **`petroMag_z`**: Petrosian magnitude measurement in z-band, assuming the object is an extended souce
* **`petroRad_r`**: size measurement of the object in r-band in arc seconds

Each of these 11 measurements also has an associated error.

## How to Obtain the Dataset Yourself

If you would like to obtain the dataset yourself, create an account on the <a href="http://skyserver.sdss.org/CasJobs/" target="_blank">SDSS CasJobs</a> site and submit the following SQL query to the DR12 catalog:

```
-- SQL Query for DR12
SELECT
    p.ra, p.dec,

    CASE s.class WHEN 'GALAXY' THEN 'Galaxy'
                 WHEN 'STAR' THEN 'Star'
                 WHEN 'QSO' THEN 'Quasar'
                 END AS class,
    
    s.subclass,
    s.z AS redshift,
    s.zErr AS redshiftErr,
    s.zWarning,
    
    p.type, p.clean, p.flags, p.probPSF,

    p.psfMag_u, p.psfMagErr_u,
    p.psfMag_g, p.psfMagErr_g,
    p.psfMag_r, p.psfMagErr_r,
    p.psfMag_i, p.psfMagErr_i,
    p.psfMag_z, p.psfMagErr_z,

    p.petroMag_u, p.petroMagErr_u,
    p.petroMag_g, p.petroMagErr_g,
    p.petroMag_r, p.petroMagErr_r,
    p.petroMag_i, p.petroMagErr_i,
    p.petroMag_z, p.petroMagErr_z,
    
    p.extinction_u, p.extinction_g, p.extinction_r, p.extinction_i, p.extinction_z,

    p.petroRad_r, p.petroRadErr_r

FROM PhotoObj AS p
    LEFT JOIN SpecObj AS s
    ON s.bestobjid = p.objid
```

### Select All Photometric Measurments

```
SELECT
    -- right ascension and declination in degrees
    p.ra, p.dec,

    -- class of object, expert opinion (galaxy, star, or quasar)
    CASE s.class WHEN 'GALAXY' THEN 'Galaxy'
                 WHEN 'STAR' THEN 'Star'
                 WHEN 'QSO' THEN 'Quasar'
                 END AS class,
    
    -- subclass of object
    s.subclass,

    -- redshift of object from spectrum with error, expert opnion
    s.z AS redshift,
    s.zErr AS redshiftErr,
    
    -- 0 if spectrum is ok
    s.zWarning,

    -- PSF magnitude measurements in 5 bands (ugriz) with error, assuming object is a point souce
    p.psfMag_u, p.psfMagErr_u,
    p.psfMag_g, p.psfMagErr_g,
    p.psfMag_r, p.psfMagErr_r,
    p.psfMag_i, p.psfMagErr_i,
    p.psfMag_z, p.psfMagErr_z,

    -- Petrosian magnitude measurements in 5 bands (ugriz) with error, assuming object is an extended souce
    p.petroMag_u, p.petroMagErr_u,
    p.petroMag_g, p.petroMagErr_g,
    p.petroMag_r, p.petroMagErr_r,
    p.petroMag_i, p.petroMagErr_i,
    p.petroMag_z, p.petroMagErr_z,
    
    -- extinction values
    p.extinction_u, p.extinction_g, p.extinction_r, p.extinction_i, p.extinction_z,

    -- size measurement in r-band in arc seconds
    p.petroRad_r, p.petroRadErr_r

FROM PhotoObj AS p
    LEFT JOIN SpecObj AS s
    ON s.bestobjid = p.objid

WHERE
    p.ra BETWEEN 180 AND 210
```

### Only Select the Best Labelled Data

```
SELECT
    -- right ascension and declination in degrees
    p.ra, p.dec,

    -- class of object, expert opinion (galaxy, star, or quasar)
    CASE s.class WHEN 'GALAXY' THEN 'Galaxy'
                 WHEN 'STAR' THEN 'Star'
                 WHEN 'QSO' THEN 'Quasar'
                 END AS class,
    
    -- subclass of object
    s.subclass,

    -- redshift of object from spectrum with error, expert opnion
    s.z AS redshift,
    s.zErr AS redshiftErr,
    
    -- 0 if spectrum is ok
    s.zWarning,

    -- PSF magnitude measurements in 5 bands (ugriz) with error, assuming object is a point souce
    p.psfMag_u, p.psfMagErr_u,
    p.psfMag_g, p.psfMagErr_g,
    p.psfMag_r, p.psfMagErr_r,
    p.psfMag_i, p.psfMagErr_i,
    p.psfMag_z, p.psfMagErr_z,

    -- Petrosian magnitude measurements in 5 bands (ugriz) with error, assuming object is an extended souce
    p.petroMag_u, p.petroMagErr_u,
    p.petroMag_g, p.petroMagErr_g,
    p.petroMag_r, p.petroMagErr_r,
    p.petroMag_i, p.petroMagErr_i,
    p.petroMag_z, p.petroMagErr_z,
    
    -- extinction values
    p.extinction_u, p.extinction_g, p.extinction_r, p.extinction_i, p.extinction_z,

    -- size measurement in r-band in arc seconds
    p.petroRad_r, p.petroRadErr_r

FROM PhotoObj AS p
   JOIN SpecObj AS s
   ON s.bestobjid = p.objid

WHERE
    -- only include objects with complete and reasonably accurate data
    p.psfMagErr_u BETWEEN 0 AND 3
    AND p.psfMagErr_g BETWEEN 0 AND 3
    AND p.psfMagErr_r BETWEEN 0 AND 3
    AND p.psfMagErr_i BETWEEN 0 AND 3
    AND p.psfMagErr_z BETWEEN 0 AND 3
    AND p.petroMagErr_u BETWEEN 0 AND 3
    AND p.petroMagErr_g BETWEEN 0 AND 3
    AND p.petroMagErr_r BETWEEN 0 AND 3
    AND p.petroMagErr_i BETWEEN 0 AND 3
    AND p.petroMagErr_z BETWEEN 0 AND 3
    AND p.petroRadErr_r BETWEEN 0 AND 3
    AND s.zErr BETWEEN 0 AND 0.1
    AND s.zWarning = 0    -- spectrum is ok
```

## Subclass

In the raw dataset, the `subclass` column is not formatted in a uniform way. Before we can work with it, we need to do a bit of cleaning up.

In [29]:
# remove null references
sdss['subclass'].replace('null', '', inplace=True)

# remove HD catalog number (stored in brackets)
sdss['subclass'].replace(r'\s*\(\d+\)\s*', '', regex=True, inplace=True)